In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

D:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def cnn(x):
    x_image=tf.reshape(x, [-1, 28, 28, 1])
    #Convolutional LAYER 1
    #
    #Initialising weights and biases
    d_w1=tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.2))
    d_b1=tf.Variable(tf.constant(0.1,shape=[32]))
    #Convolving
    d1=tf.nn.conv2d(input=x_image,filter=d_w1,strides=[1,1,1,1],padding="SAME")
    #Adding Biases
    d1+=d_b1
    #Non Linear activation layer
    d1=tf.nn.relu(d1)
    #Using max pool for pooling layer
    d1=tf.nn.max_pool(d1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    #Convolutional LAYER 2
    d_w2=tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.2))
    d_b2=tf.Variable(tf.constant(0.1,shape=[64]))
    d2=tf.nn.conv2d(input=d1,filter=d_w2,strides=[1,1,1,1],padding="SAME")
    d2+=d_b2
    d2=tf.nn.relu(d2)
    d2=tf.nn.max_pool(d2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    #Fully Connected Layer 1
    d_w3=tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.2))
    d_b3=tf.Variable(tf.constant(0.1,shape=[1024]))
    d3=tf.reshape(d2,[-1,7*7*64])
    d3=tf.matmul(d3,d_w3)
    d3+=d_b3
    d3=tf.nn.relu(d3)
    keep_prob=tf.placeholder(tf.float32)
    fc1_dropout=tf.nn.dropout(d3,keep_prob)
    #Fully Connected Layer 2
    d_w4=tf.Variable(tf.truncated_normal([1024,10],stddev=0.2))
    d_b4=tf.Variable(tf.constant(0.1,shape=[10]))
    y_conv=tf.matmul(fc1_dropout,d_w4)+d_b4
    
    return y_conv, keep_prob

In [ ]:
#create the model
x=tf.placeholder(tf.float32,[None,784])
#Defining loss and optimiser
y_=tf.placeholder(tf.float32,[None,10])
#Building graph
y_conv, keep_prob=cnn(x)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_,logits=y_conv))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))